# Final report, v-AMOS Team

## Introduction
The following notebook is intended as a report for the final project of the course "Data Science" (Prof. Peroni), consisting in a Python query processor for graph and relational databases. It was developed by the team "v-AMOS", composed by four students of the MA "Digital Humanities and Digital Humanities" at University of Bologna: [Amelia Lamaregese](mailto:amelia.lamargese@studio.unibo.it), [Olga Pagnotta](mailto:olga.pagnotta@studio.unibo.it), [Manuele Veggi](mailto:manuele.veggi@studio.unibo.it) e [Sara Vellone](mailto:sara.vellone@studio.unibo.it).

The specific guidelines of the project are available at the following [page](https://github.com/comp-data/2021-2022/tree/main/docs/project). 

![workflow](imgJupiNB/workflow.png)

## Data Model

The databases contain information on different academic publications. The entirety of the pieces of information provided can be organized within a data model and graphically visualized through UML.

![classes data model](imgJupiNB/umlClasses.png)

One of the first task hence concerned the translation of the model into a proper Python code. Following the syntax of classes declaration, we started defining properties and methods *IdentifiableEntity*, as all the other classes are its subclasses. 

In [17]:
class IdentifiableEntity:
    def __init__(self, identifier):
        self.identifier = identifier

    def getIds(self):
        return list(self.identifier)

Later on, we proceeded the subclasses *Person* and *Organization*:

In [18]:
class Person(IdentifiableEntity):
    def __init__(self, identifier, givenName, familyName):
        super().__init__(identifier)
        self.givenName = givenName
        self.familyName = familyName

    def getGivenName(self):
        return self.givenName

    def getFamilyName(self):
        return self.familyName


class Organization(IdentifiableEntity):
    def __init__(self, identifier, name):
        super().__init__(identifier)
        self.name = name

    def getName(self):
        return self.name

We also defined *Publication* and its subclasses (*JournalArticle*, *BookChapter*, *ProceedingsPaper*).

In [19]:
class Publication(IdentifiableEntity):
    def __init__(self, identifier, publicationYear, title, cited, authors, publicationVenue):
        super().__init__(identifier)
        self.publicationYear = publicationYear
        self.title = title
        self.cited = cited
        self.authors = authors
        self.publicationVenue = publicationVenue

    def getPublicationYear(self):
        return self.publicationYear

    def getTitle(self):
        return self.title

    def getCitedPublications(self):
        return self.cited 

    def getPublicationVenue(self):
        return self.publicationVenue

    def getAuthors(self):
        return self.authors  


class JournalArticle(Publication):
    def __init__(self, identifier, publicationYear, title, cited, authors, publicationVenue, issue, volume):
        super().__init__(identifier, publicationYear, title, cited, authors, publicationVenue)
        self.issue = issue
        self.volume = volume

    def getIssue(self):
        return self.issue

    def getVolume(self):
        return self.volume


class BookChapter(Publication):
    def __init__(self, identifier, publicationYear, title, cited, authors, publicationVenue, chapterNumber):
        super().__init__(identifier, publicationYear, title, cited, authors, publicationVenue)
        self.chapterNumber = chapterNumber

    def getChapterNumber(self):
        return self.chapterNumber


class ProceedingsPaper(Publication):
    pass

Lastly, we declared *Venue* and the subclasses *Journal*, *Book* and *Proceedings*.

In [20]:
class Venue(IdentifiableEntity):
    def __init__(self, identifier, title, publisher):
        super().__init__(identifier)
        self.title = title
        self.publisher = publisher

    def getTitle(self):
        return self.title

    def getPublisher(self):
        return self.publisher


class Journal(Venue):
    pass


class Book(Venue):
    pass


class Proceedings(Venue):
    def __init__(self, identifier, title, publisher, event):
        super().__init__(identifier, title, publisher)
        self.event = event

    def getEvent(self):
        return self.event

## Additional classes

The classes defining the functioning of the query processor have been described in the following UML model:

![imgJupiNB/umlQueryPro.png](imgJupiNB/umlQueryPro.png)

To complete the transaltion in Python, the workload was splitted in two halves: firstly, the creation and the query software for the relational database was created, then the same functionalities was adapted to the graph database. 

### Relational database

The handling of the relational database is handled by the the class **RelationalProcessor** and its two subclasses, *RelationalDataProcessor* and *RelationalProcessor*. The former one is defined through file path (below mentioned as *dbPath*) and by the two methods *getdbPath* and *setdbPath* and has been defined as follows:

In [ ]:
class RelationalProcessor:
    def __init__(self, dbPath):
        self.dbPath = dbPath

    def getDbPath(self):  
        return self.dbPath

    def setDbPath(self, path):
        if path != '': 
            self.dbPath = path
            return True 
        else:
            return False

The subclass **RelationalDataProcessor** is encharged of the creation of the database starting from the two input files. Given that these two inputs can be either in CSV or JSON format, the method *uploadData* needs to be modeled in two halves, one of each focused on the treatment of a specific file format. These first lines commit to *dbPath* the dataframe *General*, populated with the entries of CSV file.

In [ ]:
class RelationalDataProcessor(RelationalProcessor):
    def __init__(self, dbPath):
      super().__init__(dbPath)

    def uploadData(self, path):
        if path != '':
            if path.endswith(".csv"):
                with connect(self.dbPath) as con:
                    CSVDf = read_csv(path, keep_default_na=False)
                    CSVDf.to_sql("General", con, if_exists="replace", index=False)
                    con.commit()
                    return True

On the contrary, the JSON file has a very complex and nested structure. It is hence difficult to visualize simultaneously all the information in a sole table. The four main keys (`"authors"`, `"venues_id"`, `"references"` and `"publishers"`) allow to create four distinct dataframe and each of them has been populated with an *ad hoc* algorithm, able to take into account the peculiarities of the considered dataset. 

In [ ]:
            elif path.endswith(".json"):
                with connect(self.dbPath) as con:
                    with open(path, "r", encoding="utf-8") as f:
                        json_doc = load(f)

                    # ========== AUTHOR and PUBLICATION =======
                    authors = json_doc['authors']
                    rows_author = []
                    rows_first = []
                    for doi in authors:
                        data_row = authors[doi]
                        for row in data_row:
                            rows_author.append(row)
                        for id in range(len(authors[doi])):
                            row = [doi, id+1]
                            rows_first.append(row)
                    df1 = pd.DataFrame(rows_author)
                    df2 = pd.DataFrame(rows_first); df2.columns = ["doi", "coauthor no."]
                    author_pubDf = df2.join(df1)
                    author_pubDf.to_sql("Author", con, if_exists="replace", index=False)

                    # ========== VENUES =======================  
                    venues = json_doc['venues_id']
                    rows_ven = []
                    rows_first = []
                    for doi in venues:
                        data_row = venues[doi]
                        for idx, item_row in enumerate(data_row): 
                            rows_ven.append(item_row)
                            idno = idx + 1
                            row = [doi, idno]
                            rows_first.append(row)
                    df1 = pd.DataFrame(rows_ven); df1.columns = ["id"]
                    df2 = pd.DataFrame(rows_first); df2.columns = ["doi", "id no."]
                    venueDf = df2.join(df1)
                    venueDf.to_sql("Venue", con, if_exists="replace", index=False)

                    # ========== REFERENCES ===================
                    references = json_doc['references']
                    rows_ref = []
                    rows_first = []
                    for doi in references:
                        data_row = references[doi]
                        for row in data_row:
                            rows_ref.append(row)
                        for id in range(len(references[doi])):
                            row = [doi, id]
                            rows_first.append(row)
                    df1 = pd.DataFrame(rows_ref); df1.columns = ["doi mention"]
                    df2 = pd.DataFrame(rows_first); df2.columns = ["doi", "reference no."]
                    refDf = df2.join(df1)
                    refDf.to_sql("References", con, if_exists="replace", index=False)
                    
                    # ========== PUBLISHERS ===================
                    publishers = json_doc['publishers']
                    rowsID = []
                    rowsName = []
                    for cross_ref in publishers:
                        data_row = publishers[cross_ref]
                        rowsID.append(data_row["id"])
                        rowsName.append(data_row["name"])
                    data_tuples = list(zip(rowsID,rowsName))
                    publisherDf = pd.DataFrame(data_tuples, columns=['id','name'])
                    publisherDf.to_sql("Publisher", con, if_exists="replace", index=False)
                    con.commit()
                    return True